In [51]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from collections import Counter
import warnings
warnings.filterwarnings("ignore")

In [52]:
train_input = pd.read_csv("C:\\Credit_Risk_Train_data.csv")
test_input = pd.read_csv("C:\\Credit_Risk_Validate_data.csv")

In [53]:
print(train_input.columns)
print(test_input.columns)

Index(['Loan_ID', 'Gender', 'Married', 'Dependents', 'Education',
       'Self_Employed', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area', 'Loan_Status'],
      dtype='object')
Index(['Loan_ID', 'Gender', 'Married', 'Dependents', 'Education',
       'Self_Employed', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area', 'outcome'],
      dtype='object')


In [54]:
# the last column has a different name in both
# lets make the names same and merge them together
# so that we can fill the missing values simultaneously
test_input.rename(columns = {"outcome":"Loan_Status"},inplace=True)

In [55]:
data_all= pd.concat([train_input, test_input],axis=0)
data_all.shape

(981, 13)

In [56]:
print(data_all.tail())
data_all.reset_index(inplace =True,drop =True)
# reset index else merging will have issues

      Loan_ID Gender Married Dependents     Education Self_Employed  \
362  LP002971   Male     Yes         3+  Not Graduate           Yes   
363  LP002975   Male     Yes          0      Graduate            No   
364  LP002980   Male      No          0      Graduate            No   
365  LP002986   Male     Yes          0      Graduate            No   
366  LP002989   Male      No          0      Graduate           Yes   

     ApplicantIncome  CoapplicantIncome  LoanAmount  Loan_Amount_Term  \
362             4009             1777.0       113.0             360.0   
363             4158              709.0       115.0             360.0   
364             3250             1993.0       126.0             360.0   
365             5000             2393.0       158.0             360.0   
366             9200                0.0        98.0             180.0   

     Credit_History Property_Area Loan_Status  
362             1.0         Urban           Y  
363             1.0         Urban     

In [57]:
data_all.isnull().sum() # gives the missing value

Loan_ID               0
Gender               24
Married               3
Dependents           25
Education             0
Self_Employed        55
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           27
Loan_Amount_Term     20
Credit_History       79
Property_Area         0
Loan_Status           0
dtype: int64

In [58]:
data_all.shape # read the description of each column from the word document

(981, 13)

In [59]:
# before proceeding to Model Building, lets fill the missing values

In [60]:
Counter(data_all['Gender'])

Counter({'Male': 775, 'Female': 182, nan: 24})

In [61]:
# Lets fill them by Male
print(data_all[data_all['Gender'].isnull()].index.tolist())
# these rows are null for gender
# lets fill them with Model of Gender i.e Male
gender_null=data_all[data_all['Gender'].isnull()].index.tolist()

[23, 126, 171, 188, 314, 334, 460, 467, 477, 507, 576, 588, 592, 636, 665, 720, 752, 823, 845, 859, 893, 910, 917, 932]


In [62]:
data_all['Gender'].iloc[gender_null]='Male'

In [63]:
# cheeck if filed
print(sum(data_all['Gender'].isnull())) #ok done
Counter(data_all['Gender'])

0


Counter({'Female': 182, 'Male': 799})

In [64]:
#lets fill Married now
print(Counter(data_all['Married'])) #most are married

Counter({'Yes': 631, 'No': 347, nan: 3})


In [65]:
married_null=data_all[data_all['Married'].isnull()].index.tolist()
married_null

[104, 228, 435]

In [66]:
data_all['Married'].iloc[married_null]='Yes'

In [67]:
data_all.isnull().sum()

Loan_ID               0
Gender                0
Married               0
Dependents           25
Education             0
Self_Employed        55
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           27
Loan_Amount_Term     20
Credit_History       79
Property_Area         0
Loan_Status           0
dtype: int64

In [68]:
Counter(data_all['Dependents'])

Counter({'0': 545, '1': 160, '2': 160, '3+': 91, nan: 25})

In [69]:
pd.crosstab(data_all['Married'],data_all['Dependents'].isnull())

Dependents,False,True
Married,,
No,338,9
Yes,618,16


In [70]:
pd.crosstab(data_all['Dependents'],data_all['Married'])

Married,No,Yes
Dependents,,
0,276,269
1,36,124
2,14,146
3+,12,79


In [71]:
# for the bachelors,lets fill the missing dependents as 0
# lets find the indexof all rows with Dependents missing and Married NO
bachelor_nulldependent = data_all[(data_all['Married']=="No") & 
                                 (data_all['Dependents'].isnull())].index.tolist()
print(bachelor_nulldependent)

[293, 332, 355, 597, 684, 752, 879, 916, 926]


In [72]:
data_all['Dependents'].iloc[bachelor_nulldependent] ='0'

In [73]:
Counter(data_all['Dependents'])

Counter({'0': 554, '1': 160, '2': 160, '3+': 91, nan: 16})

In [74]:
# for the remaining 16 missing dependents
# lets see how many Male & Female dependents have
pd.crosstab(data_all['Gender'],data_all['Dependents'])

Dependents,0,1,2,3+
Gender,,,,
Female,127,32,13,9
Male,427,128,147,82


In [75]:
#so female have less dependents
# lets see the gender of 16 missing dependents
data_all['Gender'].iloc[data_all[data_all['Dependents'].isnull()].index.tolist()]

102      Male
104      Male
120      Male
226      Male
228      Male
301      Male
335      Male
346      Male
435    Female
517      Male
571      Male
660      Male
725      Male
816      Male
861      Male
865      Male
Name: Gender, dtype: object

In [76]:
pd.crosstab((data_all['Gender']=='Male')&(data_all['Married']=='Yes'),data_all['Dependents'])

Dependents,0,1,2,3+
row_0,,,,
False,318,48,23,15
True,236,112,137,76


In [77]:
# lets fill the dependent with 1
data_all['Dependents'].iloc[data_all[data_all['Dependents'].isnull()].index.tolist()]='1'

In [78]:
data_all.isnull().sum()

Loan_ID               0
Gender                0
Married               0
Dependents            0
Education             0
Self_Employed        55
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           27
Loan_Amount_Term     20
Credit_History       79
Property_Area         0
Loan_Status           0
dtype: int64

In [79]:
Counter(data_all['Self_Employed'])

Counter({'No': 807, 'Yes': 119, nan: 55})

In [80]:
self_emp_null = data_all[data_all['Self_Employed'].isnull()].index.tolist()

In [81]:
# fill missing selfemployed with NO
data_all['Self_Employed'].iloc[self_emp_null]='No'

In [82]:
data_all.isnull().sum()

Loan_ID               0
Gender                0
Married               0
Dependents            0
Education             0
Self_Employed         0
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           27
Loan_Amount_Term     20
Credit_History       79
Property_Area         0
Loan_Status           0
dtype: int64

In [83]:
# To check if any row with both LoanAmount and Loan_Amount_Term as NAN
pd.crosstab(data_all['LoanAmount'].isnull(),data_all['Loan_Amount_Term'].isnull())

Loan_Amount_Term,False,True
LoanAmount,,
False,934,20
True,27,0


In [84]:
pd.crosstab(data_all['LoanAmount'].isnull(),data_all['Loan_Amount_Term'])

Loan_Amount_Term,6.0,12.0,36.0,60.0,84.0,120.0,180.0,240.0,300.0,350.0,360.0,480.0
LoanAmount,,,,,,,,,,,,
False,1,2,3,3,7,4,64,7,20,1,800,22
True,0,0,0,0,0,0,2,1,0,0,23,1


In [85]:
data_all.groupby(data_all['Loan_Amount_Term'])['LoanAmount'].mean()

Loan_Amount_Term
6.0       95.000000
12.0     185.500000
36.0     117.666667
60.0     139.666667
84.0     121.142857
120.0     36.750000
180.0    131.125000
240.0    128.857143
300.0    166.250000
350.0    133.000000
360.0    144.420000
480.0    137.181818
Name: LoanAmount, dtype: float64

In [86]:
# lets fill the misssing values in LoanAmount
# with the mean of the respective Loan_Term
# we see that 180 & 240 has the almost same loan amount 128-131
# & 360 has hign i.e 144
# so lets fill only 360 by 144 & all remaining by 130
data_all['LoanAmount'][(data_all['LoanAmount'].isnull())&
                      (data_all['Loan_Amount_Term']==360)]=144
data_all['LoanAmount'][(data_all['LoanAmount'].isnull())&
                      (data_all['Loan_Amount_Term']==480)]=137

In [87]:
data_all['LoanAmount'][(data_all['LoanAmount'].isnull())]=130

In [88]:
# Lets fill Loan Amount Term
(data_all['Loan_Amount_Term']).value_counts()

360.0    823
180.0     66
480.0     23
300.0     20
240.0      8
84.0       7
120.0      4
36.0       3
60.0       3
12.0       2
350.0      1
6.0        1
Name: Loan_Amount_Term, dtype: int64

In [89]:
# Lets fill the loan Tenure by the mode i.e360
data_all['Loan_Amount_Term'][data_all['Loan_Amount_Term'].isnull()]=360

In [90]:
data_all.isnull().sum()

Loan_ID               0
Gender                0
Married               0
Dependents            0
Education             0
Self_Employed         0
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount            0
Loan_Amount_Term      0
Credit_History       79
Property_Area         0
Loan_Status           0
dtype: int64

In [91]:
data_all['Credit_History'].value_counts()

1.0    754
0.0    148
Name: Credit_History, dtype: int64

In [92]:
pd.crosstab(data_all['Gender'],data_all['Credit_History'])
# Gender makes no difference

Credit_History,0.0,1.0
Gender,,
Female,30,135
Male,118,619


In [93]:
pd.crosstab(data_all['Self_Employed'],data_all['Credit_History'])
# Self_Employed makes no difference

Credit_History,0.0,1.0
Self_Employed,,
No,134,658
Yes,14,96


In [94]:
pd.crosstab(data_all['Education'],data_all['Credit_History'])
# Education makes no difference

Credit_History,0.0,1.0
Education,,
Graduate,106,596
Not Graduate,42,158


In [95]:
pd.crosstab(data_all['Married'],data_all['Credit_History'])
# Married makes no difference

Credit_History,0.0,1.0
Married,,
No,56,263
Yes,92,491


In [96]:
data_all['Credit_History'][data_all['Credit_History'].isnull()]=1

In [97]:
data_all.isnull().sum()

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

In [99]:
import pickle
q=open("data_all.pkl","bw")
pickle.dump(data_all,q)
q.close()